# Autonomous Research Agent 🤖

A demonstration of core Agentic AI principles: **Tool Use** and **Multi-step Reasoning**.

This agent autonomously researches a user-provided topic by:
1.  Using the **Tavily Search API** (Tool Use) to find recent, relevant information.
2.  Synthesizing the search results with an **OpenAI Assistant** (Multi-step Reasoning).
3.  Generating a structured, professional report.

This project mirrors the capabilities of AI employees built by companies.

In [ ]:
# Install required packages. Restart the runtime if prompted.
!pip install openai tavily-python
!pip install google-generativeai
# Import the libraries
from openai import OpenAI
from tavily import TavilyClient
import os
from textwrap import fill

In [ ]:
# @title Enter your API Keys (Updated for Gemini)
from google.colab import userdata
import google.generativeai as genai

# Read the keys from secrets. We only need Tavily now.
tavily_api_key = userdata.get('TAVILY_API_KEY')
google_api_key = userdata.get('GEMINI_API_KEY') # You'll add this new secret

# Initialize the clients
tavily_client = TavilyClient(api_key=tavily_api_key)
genai.configure(api_key=google_api_key)

# Initialize the Gemini model
model = genai.GenerativeModel('gemini-1.5-flash') # Fast and good for this task

print("API clients initialized successfully!")

API clients initialized successfully!


In [ ]:
def research_topic(topic, max_results=5):
  """
  An AI agent function that uses the Tavily API to perform web search.

  Args:
    topic (str): The topic to research.
    max_results (int): Number of search results to return.

  Returns:
    str: A concatenated string of all the search results content.
  """
  print(f"🔍 Researching: '{topic}'...")
  try:
    # Perform the search using Tavily's `search` method
    search_results = tavily_client.search(query=topic, max_results=max_results, include_answer=True)
    # Check if we have results
    if not search_results['results']:
      return "No relevant information found in web search. Please try a different query."

    # Combine the content of all results into a single context string
    context = "\n\n".join([f"[Source {i+1}: {result['url']}]\n{result['content']}" for i, result in enumerate(search_results['results'])])

    print(f"✅ Found {len(search_results['results'])} sources.")
    return context

  except Exception as e:
    return f"An error occurred during research: {str(e)}"

In [ ]:
def generate_report(topic, context):
  """
  An AI agent function that uses Google Gemini to synthesize research into a report.
  """
  print("🧠 Synthesizing research into a report using Gemini...")

  # Create the prompt for Gemini
  prompt = f"""You are an expert researcher and technical writer. Your task is to synthesize the provided research context into a well-structured, comprehensive, and insightful report. Write in a professional tone. Include key findings, trends, and potential implications. Do not make up information outside of the provided context.

  Please write a detailed report on the following topic:
  **{topic}**

  Here is the context gathered from a web search:

  {context}
  """

  try:
    # Generate the content
    response = model.generate_content(prompt)
    report_content = response.text
    print("✅ Report generated!")
    return report_content
  except Exception as e:
    return f"An error occurred during report generation: {str(e)}"

In [ ]:
# @title Run the Autonomous Research Agent

# 1. Define your research topic
topic = "Latest advancements in solid-state batteries for electric vehicles" # @param {type:"string"}

# 2. Run the research function (Tool Use)
print("=" * 50)
print("🤖 Autonomous Research Agent Starting...")
print("=" * 50)
research_context = research_topic(topic, max_results=5)

# 3. Generate the final report (Reasoning)
final_report = generate_report(topic, research_context)

# 4. Display and Save the results
print("=" * 50)
print("📄 FINAL REPORT")
print("=" * 50)
print(final_report)

# Optional: Save to a file in Colab's runtime
filename = f"Report_on_{topic.replace(' ', '_')}.txt"
with open(filename, 'w') as f:
    f.write(final_report)
print("\n💾 Report also saved to:", filename)

🤖 Autonomous Research Agent Starting...
🔍 Researching: 'Latest advancements in solid-state batteries for electric vehicles'...
✅ Found 5 sources.
🧠 Synthesizing research into a report using Gemini...
✅ Report generated!
📄 FINAL REPORT
## Report: Latest Advancements in Solid-State Batteries for Electric Vehicles

**1. Introduction:**

Solid-state batteries (SSBs) represent a significant advancement in energy storage technology, offering the potential to revolutionize the electric vehicle (EV) market.  This report synthesizes recent findings regarding advancements in SSB technology for EVs, focusing on key materials, development timelines, and potential implications.

**2. Key Material Advancements:**

Research and development efforts are actively exploring various solid-state electrolytes.  Current focus includes three primary material categories: sulfide, oxide, and polymer-based electrolytes (Source 3).  Each material presents unique advantages and challenges regarding ionic conductiv

## Conclusion & Lessons Learned

This project successfully demonstrates two core principles of Agentic AI:
*   **Tool Use:** Integrating and executing calls to the Tavily Search API.
*   **Multi-step Reasoning:** Orchestrating a sequential workflow (Research -> Synthesis -> Report Generation).

**Key Technical Challenges Overcome:**
1.  **Model Compatibility:** Adapted from OpenAI's `gpt-4-turbo` to Google's `gemini-1.5-flash` to resolve API access issues.
2.  **Error Handling:** Implemented basic try/except blocks to gracefully handle potential API errors.
3.  **Context Management:** Successfully piped large volumes of text from the search API into the LLM context window for synthesis.

This architecture is a foundation for building more complex agents capable of even more autonomous task execution.